In [1]:
import sqlite3
from datetime import datetime

class ExpenseTracker:
    def __init__(self, db_name="expenses.db"):
        self.conn = sqlite3.connect(db_name)
        self.create_table()

    def create_table(self):
        query = """
        CREATE TABLE IF NOT EXISTS expenses (
            id INTEGER PRIMARY KEY,
            amount REAL,
            description TEXT,
            category TEXT,
            date TEXT
        )
        """
        self.conn.execute(query)
        self.conn.commit()

    def add_expense(self, amount, description, category, date=None):
        if date is None:
            date = datetime.now().strftime("%Y-%m-%d")
        query = """
        INSERT INTO expenses (amount, description, category, date)
        VALUES (?, ?, ?, ?)
        """
        self.conn.execute(query, (amount, description, category, date))
        self.conn.commit()
        print("Expense added successfully!")

    def fetch_expenses(self):
        query = "SELECT * FROM expenses"
        cursor = self.conn.execute(query)
        rows = cursor.fetchall()
        return rows

    def close(self):
        self.conn.close()


In [2]:
import pandas as pd

class ExpenseAnalysis:
    def __init__(self, expenses):
        self.df = pd.DataFrame(expenses, columns=["ID", "Amount", "Description", "Category", "Date"])

    def monthly_summary(self):
        self.df['Date'] = pd.to_datetime(self.df['Date'])
        self.df['Month'] = self.df['Date'].dt.to_period('M')
        summary = self.df.groupby('Month')['Amount'].sum()
        return summary

    def category_wise_summary(self):
        summary = self.df.groupby('Category')['Amount'].sum()
        return summary


In [4]:
import tkinter as tk
from tkinter import messagebox

class ExpenseTrackerApp:
    def __init__(self, master):
        self.master = master
        self.master.title("Expense Tracker")
        self.tracker = ExpenseTracker()

        # GUI components
        tk.Label(master, text="Amount:").grid(row=0)
        tk.Label(master, text="Description:").grid(row=1)
        tk.Label(master, text="Category:").grid(row=2)

        self.amount_entry = tk.Entry(master)
        self.description_entry = tk.Entry(master)
        self.category_entry = tk.Entry(master)

        self.amount_entry.grid(row=0, column=1)
        self.description_entry.grid(row=1, column=1)
        self.category_entry.grid(row=2, column=1)

        self.add_button = tk.Button(master, text="Add Expense", command=self.add_expense)
        self.add_button.grid(row=3, column=1, pady=10)

    def add_expense(self):
        amount = self.amount_entry.get()
        description = self.description_entry.get()
        category = self.category_entry.get()

        try:
            self.tracker.add_expense(float(amount), description, category)
            messagebox.showinfo("Success", "Expense added successfully!")
        except ValueError:
            messagebox.showerror("Error", "Please enter a valid amount.")

def main():
    root = tk.Tk()
    app = ExpenseTrackerApp(root)
    root.mainloop()
    app.tracker.close()

if __name__ == "__main__":
    main()
